In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from open3d.t.geometry import TriangleMesh
from tqdm import tqdm 
import time
from ConcaveHulls import ConcaveHull

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#Pre-processing
dataname = "/home/chris/Code/PointClouds/data/FLIPscans/MortenPlateTopSuperCleaned500k.ply"
pcd = o3d.io.read_point_cloud(dataname)
pcd_center = pcd.get_center()
pcd.translate(-pcd_center)

#Outlier removal
nn = 16
std_multiplier = 10

filtered_pcd = pcd.remove_statistical_outlier(nn,std_multiplier)
outliers = pcd.select_by_index(filtered_pcd[1], invert = True)
outliers.paint_uniform_color([1,0,0])
filtered_pcd = filtered_pcd[0]

#Downsampling
voxel_size = 0.01
pcd_downsampled = filtered_pcd.voxel_down_sample(voxel_size=voxel_size)
print(f'number of points: {len(pcd_downsampled.points)}')

#Extract normals
nn_distance = np.mean([pcd.compute_nearest_neighbor_distance()])
radius_normals = nn_distance*4

pcd_downsampled.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normals, max_nn=16), fast_normal_computation=True)

pcd_downsampled.paint_uniform_color([0.6,0.6,0.6])

front =  [ 0.0, 0.0, 1.0 ]
lookat = [ 4.155493041143778, 3.3619307090130235, 0.041189902146896884 ]
up =  [ 0.0, 1.0, 0.0 ]
zoom = 0.61999999999999988

pcd = pcd_downsampled

# Multi-order Ransac

max_plane_idx = 1
pt_to_plane_dist = 0.4

segment_models = {}
segments = {}
main_surface_idx = 0
largest_surface_points = 0
rest = pcd

for i in range(max_plane_idx):
    #print(f'Run {i}/{max_plane_idx} started. ', end='')
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3,num_iterations=50000)
    segments[i] = rest.select_by_index(inliers)
    if len(segments[i].points) > largest_surface_points:
        largest_surface_points = len(segments[i].points) 
        main_surface_idx = i
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print('Done')

# Gap detection
main_surface_pcd = segments[main_surface_idx]
np.asarray(main_surface_pcd.points)[:, 2] = 0

points = np.asarray(main_surface_pcd.points)
pcd_tree = o3d.geometry.KDTreeFlann(main_surface_pcd)

grid_resolution = 0.1
search_radius = 0.5
neighbor_threshold = 4

min_bound = main_surface_pcd.get_min_bound()
max_bound = main_surface_pcd.get_max_bound()

x_grid = np.arange(min_bound[0], max_bound[0], grid_resolution)
y_grid = np.arange(min_bound[1], max_bound[1], grid_resolution)
z_value = np.mean([min_bound[2], max_bound[2]])

# Generate all query points at once
xv, yv = np.meshgrid(x_grid, y_grid, indexing='ij')
query_points = np.vstack((xv.ravel(), yv.ravel(), np.full_like(xv.ravel(), z_value))).T

gap_points = []

# Efficient neighbor search
for query_point in tqdm(query_points):
    k, idx, _ = pcd_tree.search_radius_vector_3d(query_point, search_radius)
    if k < neighbor_threshold:
        gap_points.append(query_point)

gap_pcd = o3d.geometry.PointCloud()
gap_pcd.points = o3d.utility.Vector3dVector(gap_points)
gap_pcd.paint_uniform_color([1, 0, 0])

#DBScan clustering

eps = 0.30
min_points = 10

labels = np.array(gap_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
print(labels[0])
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[labels]
colors[labels<0] = 0
gap_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

#Filter small clusters

points = np.asarray(gap_pcd.points)
colors = np.asarray(gap_pcd.colors)  
point_threshold = 45
mask = np.zeros(len(points), dtype=bool)
unique_labels = np.unique(labels)

filtered_pcd = o3d.geometry.PointCloud()
filtered_clusters = []
filtered_labels_unique = []
filtered_labels_vector = []

#Filter and assign
for label in unique_labels:
    if label != -1:
        cluster_indices = np.where(labels == label)[0]
        if len(cluster_indices) >= point_threshold:
            mask[cluster_indices] = True
            cluster_points = points[cluster_indices]
            filtered_labels_unique.append(label)

for label in filtered_labels_unique:
    cluster_indices = np.where(labels == label)[0]
    cluster_points = points[cluster_indices]
    filtered_clusters.append(cluster_points)
    cluster_labels = np.tile(label, (cluster_points.shape[0], 1))
    filtered_labels_vector.append(cluster_labels)

total_points = np.vstack([cluster for cluster in filtered_clusters])
filtered_labels = np.vstack([cluster_labels for cluster_labels in filtered_labels_vector]).squeeze()

#print(total_points.shape)
#print(filtered_labels.shape)

filtered_pcd.points = o3d.utility.Vector3dVector(total_points)

#Recolor the filtered pcd
max_label = filtered_labels.max()
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[filtered_labels]
filtered_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.visualization.draw_geometries([filtered_pcd], zoom=zoom, front=front, lookat=lookat, up=up)

number of points: 493486
Done


100%|██████████| 6193050/6193050 [00:41<00:00, 149859.28it/s]


Precompute neighbors.[========================================] 100%
point cloud has 103 clusters======================>] 97%
0


In [6]:
cluster_points = filtered_clusters[4]
cluster_points[:, 2] = 0
concave_hull = ConcaveHull(cluster_points, k=5, visualize=False, verbose=False)
hull = concave_hull.compute_concave_hull()